In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function
import scipy.sparse as sp
import numpy as np
from tensorflow.contrib.rnn import RNNCell
import tensorflow as tf
from tensorflow.python.platform import tf_logging as logging
import pickle as pkl
import tensorflow as tf
import pandas as pd
import numpy as np
import math
import os
import numpy.linalg as la
from sklearn.metrics import mean_squared_error,mean_absolute_error
#import matplotlib.pyplot as plt
import time
time_start=time.time()

In [2]:
# Loading of Dataset-SZ Traffic
def load_sz_data():
    sz_adj = pd.read_csv('sz_adj.csv',header=None)
    adj = np.mat(sz_adj)
    sz_tf = pd.read_csv('sz_speed.csv')
    return sz_tf, adj

data, adj = load_sz_data()

In [3]:
print(data) # Time sequence data

          90217      90218      90219      90220      90221      90222  \
0      8.471971  18.455410  20.590635  15.345258   9.585218  21.501821   
1      7.807137  15.713816  27.523695  11.087895   9.455280  17.332246   
2      8.809457   8.979647  20.280394  16.523419   8.003314  15.789483   
3     51.590372  23.631243  20.224094  15.116459   6.642644  17.575806   
4     58.770433  20.437740  20.465606  14.820217  11.344404   0.000000   
5     58.289126  10.332738  25.331018  18.485616   2.028513  10.718488   
6     61.334449  20.818480  15.839392   8.768217  13.933326  13.866124   
7     58.903144   8.291826  44.043729  15.250859   0.000000   0.000000   
8     57.210441  25.765667  18.677565   5.066051   0.000000  12.645903   
9     59.559265  16.803983  20.314610  23.393083   2.612821   0.000000   
10    60.217867  26.773576  17.684078  17.735375  12.957171   7.404007   
11    56.336405  18.866552  27.502295  18.201105  23.973238  22.215358   
12    58.511395  15.802270  15.900551 

In [4]:
print(adj) #Adjacency Matrix of Data

[[0 0 0 ... 0 0 0]
 [0 0 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 ...
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 0 0 1]
 [0 0 0 ... 1 1 0]]


In [5]:
print(data.shape)
time_len=data.shape[0] # Time sequence length
num_nodes=data.shape[1] #Number of Roads

(2976, 156)


In [6]:
# Setting the variables
output_dim=pre_len=4
seq_len=4
num_units=100
train_rate=0.8
batch_size=32

In [7]:
#Normalization : Traffic Speed Data

data1 =np.mat(data,dtype=np.float32)

max_value = np.max(data1)
data1  = data1/max_value
print(data1)

   

[[0.0980221  0.21353212 0.23823702 ... 0.19194394 0.37707424 0.        ]
 [0.09032986 0.18181142 0.31845367 ... 0.45834997 0.37705195 0.        ]
 [0.10192686 0.10389599 0.23464748 ... 0.3856561  0.43354756 0.        ]
 ...
 [0.37947276 0.141638   0.10453826 ... 0.13605069 0.1998609  0.        ]
 [0.39722532 0.18600048 0.121989   ... 0.16306393 0.16957766 0.        ]
 [0.38101357 0.14001252 0.10420388 ... 0.19054307 0.1845446  0.        ]]


In [8]:
def preprocess_data(data, time_len, rate, seq_len, pre_len):
    train_size = int(time_len * rate) #2976 *0.8 =2380
    train_data = data[0:train_size] #  [0:2380]
    test_data = data[train_size:time_len] #[2380:2976]
    print(train_data.shape,'----->',test_data.shape)

    trainX, trainY, testX, testY = [], [], [], []
    for i in range(len(train_data) - seq_len - pre_len): #(2380-4-1)=2375
        a = train_data[i: i + seq_len + pre_len] #[0:0+4+1] 
        trainX.append(a[0 : seq_len]) #a[0:4] 4 time * 156 roads
        trainY.append(a[seq_len : seq_len + pre_len]) #a[4:4+1] 1 time*156 
    for i in range(len(test_data) - seq_len -pre_len):
        b = test_data[i: i + seq_len + pre_len]
        testX.append(b[0 : seq_len])
        testY.append(b[seq_len : seq_len + pre_len])
      
    trainX1 = np.array(trainX) 
    trainY1 = np.array(trainY)
    testX1 = np.array(testX)
    testY1 = np.array(testY)
    return trainX1, trainY1, testX1, testY1

trainX, trainY, testX, testY = preprocess_data(data1, time_len, train_rate, seq_len, pre_len)

totalbatch = int(trainX.shape[0]/batch_size)
training_data_count = len(trainX)  
print('Train Test Split Details :')
print('Train x ----> ',len(trainX))
print('Train y ----> ',len(trainY))
print(trainX.shape)
print('Test x ----> ',len(testX))
print('Test y ----> ',len(testY))
print('Total Batch ----> ',totalbatch)
#print('\nTrain Sample Details :')
#print(trainX[0],'--->',trainY[0])


(2380, 156) -----> (596, 156)
Train Test Split Details :
Train x ---->  2372
Train y ---->  2372
(2372, 4, 156)
Test x ---->  588
Test y ---->  588
Total Batch ---->  74


In [9]:
def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(1 / (input_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) 
#weights =weight_variable_glorot(4, 64, name='weights')
#weights

In [10]:
class stgcnCell(RNNCell):
    """Temporal Graph Convolutional Network """

    def call(self, inputs, **kwargs):
        pass

    def __init__(self, num_units, adj, num_nodes, input_size=None,
                 act=tf.nn.tanh, reuse=None):

        super(stgcnCell, self).__init__(_reuse=reuse)
        self._act = act
        self._nodes = num_nodes
        self._units = num_units
        self._adj = []
        self._adj.append(self.calculate_laplacian(adj))

    @staticmethod
    def _build_sparse_matrix(L):
        L = L.tocoo()
        indices = np.column_stack((L.row, L.col))
        L = tf.SparseTensor(indices, L.data, L.shape)
        return tf.sparse_reorder(L)

    def calculate_laplacian(self,adj, lambda_max=1):  
        adj = self.normalized_adj(adj + sp.eye(adj.shape[0])) # normalisation(self identity matrix + adj)
        adj = sp.csr_matrix(adj) #compressed sparse matrix
        adj = adj.astype(np.float32)
        return self.sparse_to_tuple(adj)
    
    def normalized_adj(self,adj):
        adj = sp.coo_matrix(adj)
        degree = np.array(adj.sum(1)) # Degree Matrix row wise sum
        d_inv_sqrt = np.power(degree, -0.5).flatten() # D inv = Degree ^-0.5 
        d_mat_inv_sqrt = sp.diags(d_inv_sqrt) #substitution of the 1D array degree in a 2D matrix diagonals
        normalized_adj = adj.dot(d_mat_inv_sqrt).transpose().dot(d_mat_inv_sqrt).tocoo() # norm= D^-0.5 * adj * D^-0.5
        normalized_adj = normalized_adj.astype(np.float32) 
        return normalized_adj
     
    ''' def weight_variable_glorot(input_dim, output_dim, name=""):
        init_range = np.sqrt(6.0 / (input_dim + output_dim))
        initial = tf.random_uniform([input_dim, output_dim], minval=-init_range, maxval=init_range, dtype=tf.float32)
        return tf.Variable(initial,name=name) '''
    
    def sparse_to_tuple(self,mx):
        mx = mx.tocoo()
        coords = np.vstack((mx.row, mx.col)).transpose() #coordinate stacking row and column wise and transpose
        L = tf.SparseTensor(coords, mx.data, mx.shape) # mx.shape= (156,156)
        #print('shape ---->',mx.shape)
        return tf.sparse_reorder(L) #row major ordering
        
    def init_state(self,batch_size):       
        state = tf.zeros(shape=[batch_size, self._num_nodes*self._num_units], dtype=tf.float32)
        return state  
               
    @staticmethod
    def _concat(x, x_):
        x_ = tf.expand_dims(x_, 0)
        return tf.concat([x, x_], axis=0)   
    @property
    def state_size(self):
        return self._nodes * self._units

    @property
    def output_size(self):
        return self._units

    def __call__(self, inputs, state, scope=None):

        with tf.variable_scope(scope or "tgcn"):
            with tf.variable_scope("gates"):  
                value = tf.nn.sigmoid(
                    self._gc(inputs, state, 2 * self._units, bias=1.0, scope=scope))
                r, u = tf.split(value=value, num_or_size_splits=2, axis=1)
            with tf.variable_scope("candidate"):
                r_state = r * state
                c = self._act(self._gc(inputs, r_state, self._units, scope=scope))
            new_h = u * state + (1 - u) * c
        return new_h, new_h


    def _gc(self, inputs, state, output_size, bias=0.0, scope=None):
        ## inputs:(-1,num_nodes)
        inputs = tf.expand_dims(inputs, 2)
        ## state:(batch,num_node,gru_units)
        state = tf.reshape(state, (-1, self._nodes, self._units))#(-1,156,64)
        ## concat
        x_s = tf.concat([inputs, state], axis=2)
        input_size = x_s.get_shape()[2].value
        ## (num_node,input_size,-1)
        x0 = tf.transpose(x_s, perm=[1, 2, 0])  
        x0 = tf.reshape(x0, shape=[self._nodes, -1])
        
        scope = tf.get_variable_scope()
        with tf.variable_scope(scope):
            for m in self._adj:
                x1 = tf.sparse_tensor_dense_matmul(m, x0)
#                print(x1)
            x = tf.reshape(x1, shape=[self._nodes, input_size,-1])
            x = tf.transpose(x,perm=[2,0,1])
            x = tf.reshape(x, shape=[-1, input_size])
            weights = weight_variable_glorot(input_size, output_size, name='weights')
            x = tf.matmul(x, weights)  # (batch_size * self._nodes, output_size)
            biases = tf.get_variable(
                "biases", [output_size], initializer=tf.constant_initializer(bias, dtype=tf.float32))
            x = tf.nn.bias_add(x, biases)
            x = tf.reshape(x, shape=[-1, self._nodes, output_size])
            x = tf.reshape(x, shape=[-1, self._nodes * output_size])
        return x

In [11]:
def STGCN(_X, _weights, _biases):
    ###
    cell_1 = stgcnCell(num_units, adj, num_nodes=num_nodes)
    cell = tf.nn.rnn_cell.MultiRNNCell([cell_1], state_is_tuple=True)
    _X = tf.unstack(_X, axis=1)
    outputs, states = tf.nn.static_rnn(cell, _X, dtype=tf.float32)
    m = []
    for i in outputs:
        o = tf.reshape(i,shape=[-1,num_nodes,num_units])
        o = tf.reshape(o,shape=[-1,num_units])
        m.append(o)
    last_output = m[-1]
    output = tf.matmul(last_output, _weights['out']) + _biases['out']
    output = tf.reshape(output,shape=[-1,num_nodes,pre_len])
    output = tf.transpose(output, perm=[0,2,1])
    output = tf.reshape(output, shape=[-1,num_nodes])
    return output, m, states

In [12]:
inputs = tf.placeholder(tf.float32, shape=[None, seq_len, num_nodes])
labels = tf.placeholder(tf.float32, shape=[None, pre_len, num_nodes])
weights = {
    'out': tf.Variable(tf.random_normal([num_units, pre_len], mean=1.0), name='weight_o')}
biases = {
    'out': tf.Variable(tf.random_normal([pre_len]),name='bias_o')}
print(type(inputs))
pred,ttts,ttto = STGCN(inputs, weights, biases)

Instructions for updating:
Colocations handled automatically by placer.
<class 'tensorflow.python.framework.ops.Tensor'>
Instructions for updating:
This class is equivalent as tf.keras.layers.StackedRNNCells, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell, unroll=True)`, which is equivalent to this API


In [13]:
y_pred = pred

In [14]:
lambda_loss = 0.0015
Lreg = lambda_loss * sum(tf.nn.l2_loss(tf_var) for tf_var in tf.trainable_variables())
label = tf.reshape(labels, [-1,num_nodes])

In [15]:
loss = tf.reduce_mean(tf.nn.l2_loss(y_pred-label) + Lreg)

In [16]:
error = tf.sqrt(tf.reduce_mean(tf.square(y_pred-label)))

In [17]:
lr=0.001
optimizer = tf.train.AdamOptimizer(lr).minimize(loss)

In [18]:
variables = tf.global_variables()
training_epoch=1000
saver = tf.train.Saver(tf.global_variables()) #
#sess = tf.Session()
gpu_options = tf.GPUOptions(per_process_gpu_memory_fraction=0.333)
sess = tf.Session(config=tf.ConfigProto(gpu_options=gpu_options))
sess.run(tf.global_variables_initializer())
out = 'out/%s'%("STGCN")
#out = 'out/%s_%s'%(model_name,'perturbation')
path1 = '%s_%s_lr%r_batch%r_unit%r_seq%r_pre%r_epoch%r'%("STGCN","sz",lr,batch_size,num_units,seq_len,pre_len,training_epoch)
path = os.path.join(out,path1)
if not os.path.exists(path):
    os.makedirs(path)

In [19]:
def evaluation(a,b):
    rmse = math.sqrt(mean_squared_error(a,b))
    mae = mean_absolute_error(a, b)
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    r2 = 1-((a-b)**2).sum()/((a-a.mean())**2).sum()
    var = 1-(np.var(a-b))/np.var(a)
    return rmse, mae, 1-F_norm, r2, var

In [20]:
x_axe,batch_loss,batch_rmse,batch_pred = [], [], [], []
test_loss,test_rmse,test_mae,test_acc,test_r2,test_var,test_pred = [],[],[],[],[],[],[]

In [21]:
def acc(a,b):  
    F_norm = la.norm(a-b,'fro')/la.norm(a,'fro')
    train_acc=1-F_norm
    return train_acc

In [22]:
for epoch in range(training_epoch):
    print("Epoch ", epoch)
    for m in range(totalbatch):
        mini_batch = trainX[m * batch_size : (m+1) * batch_size]
        mini_label = trainY[m * batch_size : (m+1) * batch_size]
        _, loss1, rmse1, train_output = sess.run([optimizer, loss, error, y_pred],
                                                 feed_dict = {inputs:mini_batch, labels:mini_label})
        batch_loss.append(loss1)
        batch_rmse.append(rmse1 * max_value)
        train_label=np.reshape(mini_label,[-1,num_nodes])
        

     # Test completely at every epoch
    print("Accuracy ----> ", evaluation(train_label,train_output)[2])
    loss2, rmse2, test_output = sess.run([loss, error, y_pred],
                                         feed_dict = {inputs:testX, labels:testY})
    #train_label=np.reshape(trainY,[-1,num_nodes])
    #train_acc=acc(train_label,train_output)
    test_label = np.reshape(testY,[-1,num_nodes])
    rmse, mae, acc, r2_score, var_score = evaluation(test_label, test_output)
    test_label1 = test_label * max_value#Inverse normalization
    test_output1 = test_output * max_value
    test_loss.append(loss2)
    test_rmse.append(rmse * max_value)
    test_mae.append(mae * max_value)
    test_acc.append(acc)
    test_r2.append(r2_score)
    test_var.append(var_score)
    test_pred.append(test_output1)
    print(mini_label.shape,train_output.shape)
    print('Iter:{}'.format(epoch),
          'train_rmse:{:.4}'.format(batch_rmse[-1]),
          'test_loss:{:.4}'.format(loss2),
          'test_rmse:{:.4}'.format(rmse),
          'test_acc:{:.4}'.format(acc))
    if (epoch % 500 == 0):        
        saver.save(sess, path+'/model_100/TGCN_pre_%r'%epoch, global_step = epoch)
        
time_end = time.time()
print(time_end-time_start,'s')

Epoch  0
Accuracy ---->  0.2605741620063782
(32, 4, 156) (128, 156)
Iter:0 train_rmse:10.62 test_loss:2.867e+03 test_rmse:0.125 test_acc:0.2475
Epoch  1
Accuracy ---->  0.48185449838638306
(32, 4, 156) (128, 156)
Iter:1 train_rmse:7.443 test_loss:1.474e+03 test_rmse:0.0896 test_acc:0.4606
Epoch  2
Accuracy ---->  0.5251418352127075
(32, 4, 156) (128, 156)
Iter:2 train_rmse:6.821 test_loss:1.254e+03 test_rmse:0.08263 test_acc:0.5025
Epoch  3
Accuracy ---->  0.537283182144165
(32, 4, 156) (128, 156)
Iter:3 train_rmse:6.647 test_loss:1.195e+03 test_rmse:0.08069 test_acc:0.5142
Epoch  4
Accuracy ---->  0.546808511018753
(32, 4, 156) (128, 156)
Iter:4 train_rmse:6.51 test_loss:1.151e+03 test_rmse:0.07916 test_acc:0.5234
Epoch  5
Accuracy ---->  0.5562416017055511
(32, 4, 156) (128, 156)
Iter:5 train_rmse:6.374 test_loss:1.107e+03 test_rmse:0.07765 test_acc:0.5325
Epoch  6
Accuracy ---->  0.5655696392059326
(32, 4, 156) (128, 156)
Iter:6 train_rmse:6.24 test_loss:1.066e+03 test_rmse:0.0762 t

Accuracy ---->  0.6591971218585968
(32, 4, 156) (128, 156)
Iter:57 train_rmse:4.895 test_loss:713.7 test_rmse:0.06232 test_acc:0.6248
Epoch  58
Accuracy ---->  0.6600225567817688
(32, 4, 156) (128, 156)
Iter:58 train_rmse:4.884 test_loss:711.1 test_rmse:0.06221 test_acc:0.6255
Epoch  59
Accuracy ---->  0.6608453094959259
(32, 4, 156) (128, 156)
Iter:59 train_rmse:4.872 test_loss:708.6 test_rmse:0.0621 test_acc:0.6261
Epoch  60
Accuracy ---->  0.6616623401641846
(32, 4, 156) (128, 156)
Iter:60 train_rmse:4.86 test_loss:706.2 test_rmse:0.06199 test_acc:0.6268
Epoch  61
Accuracy ---->  0.662467896938324
(32, 4, 156) (128, 156)
Iter:61 train_rmse:4.848 test_loss:703.7 test_rmse:0.06188 test_acc:0.6274
Epoch  62
Accuracy ---->  0.6632523834705353
(32, 4, 156) (128, 156)
Iter:62 train_rmse:4.837 test_loss:701.3 test_rmse:0.06177 test_acc:0.6281
Epoch  63
Accuracy ---->  0.6640016436576843
(32, 4, 156) (128, 156)
Iter:63 train_rmse:4.826 test_loss:698.8 test_rmse:0.06166 test_acc:0.6287
Epoch

Accuracy ---->  0.6805890500545502
(32, 4, 156) (128, 156)
Iter:114 train_rmse:4.588 test_loss:638.3 test_rmse:0.05892 test_acc:0.6453
Epoch  115
Accuracy ---->  0.6807630062103271
(32, 4, 156) (128, 156)
Iter:115 train_rmse:4.586 test_loss:637.7 test_rmse:0.05889 test_acc:0.6454
Epoch  116
Accuracy ---->  0.6809390187263489
(32, 4, 156) (128, 156)
Iter:116 train_rmse:4.583 test_loss:637.0 test_rmse:0.05886 test_acc:0.6456
Epoch  117
Accuracy ---->  0.6811177432537079
(32, 4, 156) (128, 156)
Iter:117 train_rmse:4.581 test_loss:636.4 test_rmse:0.05883 test_acc:0.6458
Epoch  118
Accuracy ---->  0.6812996566295624
(32, 4, 156) (128, 156)
Iter:118 train_rmse:4.578 test_loss:635.8 test_rmse:0.0588 test_acc:0.646
Epoch  119
Accuracy ---->  0.6814852952957153
(32, 4, 156) (128, 156)
Iter:119 train_rmse:4.575 test_loss:635.2 test_rmse:0.05877 test_acc:0.6461
Epoch  120
Accuracy ---->  0.6816745400428772
(32, 4, 156) (128, 156)
Iter:120 train_rmse:4.573 test_loss:634.5 test_rmse:0.05874 test_ac

Accuracy ---->  0.6963107883930206
(32, 4, 156) (128, 156)
Iter:171 train_rmse:4.362 test_loss:585.0 test_rmse:0.05638 test_acc:0.6605
Epoch  172
Accuracy ---->  0.6964812874794006
(32, 4, 156) (128, 156)
Iter:172 train_rmse:4.36 test_loss:584.3 test_rmse:0.05635 test_acc:0.6607
Epoch  173
Accuracy ---->  0.6966502368450165
(32, 4, 156) (128, 156)
Iter:173 train_rmse:4.357 test_loss:583.7 test_rmse:0.05631 test_acc:0.6609
Epoch  174
Accuracy ---->  0.6968177258968353
(32, 4, 156) (128, 156)
Iter:174 train_rmse:4.355 test_loss:583.0 test_rmse:0.05628 test_acc:0.6611
Epoch  175
Accuracy ---->  0.696984201669693
(32, 4, 156) (128, 156)
Iter:175 train_rmse:4.353 test_loss:582.3 test_rmse:0.05625 test_acc:0.6613
Epoch  176
Accuracy ---->  0.697149783372879
(32, 4, 156) (128, 156)
Iter:176 train_rmse:4.35 test_loss:581.6 test_rmse:0.05622 test_acc:0.6615
Epoch  177
Accuracy ---->  0.6973147690296173
(32, 4, 156) (128, 156)
Iter:177 train_rmse:4.348 test_loss:580.9 test_rmse:0.05618 test_acc:

(32, 4, 156) (128, 156)
Iter:227 train_rmse:4.23 test_loss:549.9 test_rmse:0.05465 test_acc:0.671
Epoch  228
Accuracy ---->  0.7056914567947388
(32, 4, 156) (128, 156)
Iter:228 train_rmse:4.228 test_loss:549.3 test_rmse:0.05462 test_acc:0.6712
Epoch  229
Accuracy ---->  0.7058965563774109
(32, 4, 156) (128, 156)
Iter:229 train_rmse:4.225 test_loss:548.6 test_rmse:0.05458 test_acc:0.6714
Epoch  230
Accuracy ---->  0.7061026096343994
(32, 4, 156) (128, 156)
Iter:230 train_rmse:4.222 test_loss:547.9 test_rmse:0.05455 test_acc:0.6716
Epoch  231
Accuracy ---->  0.7063092291355133
(32, 4, 156) (128, 156)
Iter:231 train_rmse:4.219 test_loss:547.3 test_rmse:0.05452 test_acc:0.6718
Epoch  232
Accuracy ---->  0.7065163254737854
(32, 4, 156) (128, 156)
Iter:232 train_rmse:4.216 test_loss:546.6 test_rmse:0.05448 test_acc:0.672
Epoch  233
Accuracy ---->  0.7067235708236694
(32, 4, 156) (128, 156)
Iter:233 train_rmse:4.213 test_loss:546.0 test_rmse:0.05445 test_acc:0.6722
Epoch  234
Accuracy ---->  

...

Accuracy ---->  0.7404286563396454
(32, 4, 156) (128, 156)
Iter:739 train_rmse:3.729 test_loss:441.6 test_rmse:0.04894 test_acc:0.7053
Epoch  740
Accuracy ---->  0.7405125200748444
(32, 4, 156) (128, 156)
Iter:740 train_rmse:3.727 test_loss:441.5 test_rmse:0.04894 test_acc:0.7054
Epoch  741
Accuracy ---->  0.7406039834022522
(32, 4, 156) (128, 156)
Iter:741 train_rmse:3.726 test_loss:441.3 test_rmse:0.04893 test_acc:0.7054
Epoch  742
Accuracy ---->  0.7406816482543945
(32, 4, 156) (128, 156)
Iter:742 train_rmse:3.725 test_loss:441.0 test_rmse:0.04891 test_acc:0.7055
Epoch  743
Accuracy ---->  0.7406927049160004
(32, 4, 156) (128, 156)
Iter:743 train_rmse:3.725 test_loss:440.8 test_rmse:0.0489 test_acc:0.7056
Epoch  744
Accuracy ---->  0.7406679093837738
(32, 4, 156) (128, 156)
Iter:744 train_rmse:3.725 test_loss:440.8 test_rmse:0.0489 test_acc:0.7056
Epoch  745
Accuracy ---->  0.7406605780124664
(32, 4, 156) (128, 156)
Iter:745 train_rmse:3.725 test_loss:440.8 test_rmse:0.0489 test_acc

Accuracy ---->  0.7414946854114532
(32, 4, 156) (128, 156)
Iter:796 train_rmse:3.713 test_loss:439.2 test_rmse:0.04881 test_acc:0.7061
Epoch  797
Accuracy ---->  0.7415915727615356
(32, 4, 156) (128, 156)
Iter:797 train_rmse:3.712 test_loss:439.0 test_rmse:0.0488 test_acc:0.7062
Epoch  798
Accuracy ---->  0.7415618300437927
(32, 4, 156) (128, 156)
Iter:798 train_rmse:3.712 test_loss:438.8 test_rmse:0.04879 test_acc:0.7062
Epoch  799
Accuracy ---->  0.7415339946746826
(32, 4, 156) (128, 156)
Iter:799 train_rmse:3.713 test_loss:438.9 test_rmse:0.04879 test_acc:0.7062
Epoch  800
Accuracy ---->  0.7415362000465393
(32, 4, 156) (128, 156)
Iter:800 train_rmse:3.713 test_loss:438.9 test_rmse:0.0488 test_acc:0.7062
Epoch  801
Accuracy ---->  0.7415474951267242
(32, 4, 156) (128, 156)
Iter:801 train_rmse:3.712 test_loss:439.0 test_rmse:0.0488 test_acc:0.7062
Epoch  802
Accuracy ---->  0.7415623366832733
(32, 4, 156) (128, 156)
Iter:802 train_rmse:3.712 test_loss:439.0 test_rmse:0.0488 test_acc:

Accuracy ---->  0.7423550486564636
(32, 4, 156) (128, 156)
Iter:853 train_rmse:3.701 test_loss:438.2 test_rmse:0.04876 test_acc:0.7064
Epoch  854
Accuracy ---->  0.7424044907093048
(32, 4, 156) (128, 156)
Iter:854 train_rmse:3.7 test_loss:438.1 test_rmse:0.04875 test_acc:0.7065
Epoch  855
Accuracy ---->  0.7424276471138
(32, 4, 156) (128, 156)
Iter:855 train_rmse:3.7 test_loss:438.0 test_rmse:0.04875 test_acc:0.7065
Epoch  856
Accuracy ---->  0.7424428462982178
(32, 4, 156) (128, 156)
Iter:856 train_rmse:3.7 test_loss:437.9 test_rmse:0.04874 test_acc:0.7065
Epoch  857
Accuracy ---->  0.7424745857715607
(32, 4, 156) (128, 156)
Iter:857 train_rmse:3.699 test_loss:437.7 test_rmse:0.04873 test_acc:0.7066
Epoch  858
Accuracy ---->  0.7425171732902527
(32, 4, 156) (128, 156)
Iter:858 train_rmse:3.699 test_loss:437.5 test_rmse:0.04872 test_acc:0.7067
Epoch  859
Accuracy ---->  0.7425612211227417
(32, 4, 156) (128, 156)
Iter:859 train_rmse:3.698 test_loss:437.2 test_rmse:0.0487 test_acc:0.7068

Accuracy ---->  0.7423494160175323
(32, 4, 156) (128, 156)
Iter:910 train_rmse:3.701 test_loss:436.5 test_rmse:0.04867 test_acc:0.707
Epoch  911
Accuracy ---->  0.7423594295978546
(32, 4, 156) (128, 156)
Iter:911 train_rmse:3.701 test_loss:436.5 test_rmse:0.04866 test_acc:0.707
Epoch  912
Accuracy ---->  0.7423738539218903
(32, 4, 156) (128, 156)
Iter:912 train_rmse:3.701 test_loss:436.5 test_rmse:0.04866 test_acc:0.707
Epoch  913
Accuracy ---->  0.7423898875713348
(32, 4, 156) (128, 156)
Iter:913 train_rmse:3.7 test_loss:436.4 test_rmse:0.04866 test_acc:0.707
Epoch  914
Accuracy ---->  0.7424054741859436
(32, 4, 156) (128, 156)
Iter:914 train_rmse:3.7 test_loss:436.4 test_rmse:0.04866 test_acc:0.707
Epoch  915
Accuracy ---->  0.7424197196960449
(32, 4, 156) (128, 156)
Iter:915 train_rmse:3.7 test_loss:436.4 test_rmse:0.04866 test_acc:0.7071
Epoch  916
Accuracy ---->  0.7424309849739075
(32, 4, 156) (128, 156)
Iter:916 train_rmse:3.7 test_loss:436.3 test_rmse:0.04865 test_acc:0.7071
Ep

Accuracy ---->  0.7427865564823151
(32, 4, 156) (128, 156)
Iter:967 train_rmse:3.695 test_loss:435.9 test_rmse:0.04863 test_acc:0.7072
Epoch  968
Accuracy ---->  0.7428770363330841
(32, 4, 156) (128, 156)
Iter:968 train_rmse:3.693 test_loss:435.7 test_rmse:0.04862 test_acc:0.7073
Epoch  969
Accuracy ---->  0.7430109083652496
(32, 4, 156) (128, 156)
Iter:969 train_rmse:3.691 test_loss:435.2 test_rmse:0.04859 test_acc:0.7074
Epoch  970
Accuracy ---->  0.7426748275756836
(32, 4, 156) (128, 156)
Iter:970 train_rmse:3.696 test_loss:435.1 test_rmse:0.04859 test_acc:0.7075
Epoch  971
Accuracy ---->  0.7426259219646454
(32, 4, 156) (128, 156)
Iter:971 train_rmse:3.697 test_loss:435.9 test_rmse:0.04863 test_acc:0.7072
Epoch  972
Accuracy ---->  0.742777407169342
(32, 4, 156) (128, 156)
Iter:972 train_rmse:3.695 test_loss:435.9 test_rmse:0.04863 test_acc:0.7072
Epoch  973
Accuracy ---->  0.7428498268127441
(32, 4, 156) (128, 156)
Iter:973 train_rmse:3.694 test_loss:435.8 test_rmse:0.04863 test_a

In [23]:
b = int(len(batch_rmse)/totalbatch)
batch_rmse1 = [i for i in batch_rmse]
train_rmse = [(sum(batch_rmse1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]
batch_loss1 = [i for i in batch_loss]
train_loss = [(sum(batch_loss1[i*totalbatch:(i+1)*totalbatch])/totalbatch) for i in range(b)]

index = test_rmse.index(np.min(test_rmse))
test_result = test_pred[index]
var = pd.DataFrame(test_result)
var.to_csv(path+'/newtest_result.csv',index = False,header = False)
#plot_result(test_result,test_label1,path)
#plot_error(train_rmse,train_loss,test_rmse,test_acc,test_mae,path)

In [24]:
print("Testing:")
print('min_rmse:%r'%(np.min(test_rmse)),
      'min_mae:%r'%(test_mae[index]),
      'max_acc:%r'%(test_acc[index]),
      'r2:%r'%(test_r2[index]),
      'var:%r'%test_var[index])

Testing:
min_rmse:4.199411593474004 min_mae:2.9005845 max_acc:0.7074674665927887 r2:0.8382712006568909 var:0.83915875852108


In [25]:
print("Training:")
rmse, mae, acc, r2_score, var_score = evaluation(train_label,train_output)
print('min_rmse:%r'%(rmse),
      'min_mae:%r'%(mae),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score)) 

Training:
min_rmse:0.042706044114408065 min_mae:0.029557265 max_acc:0.7430389821529388 r2:0.8666488379240036 var:0.8672683238983154


In [26]:
#inverse normalisation
print('min_rmse:%r'%(rmse*max_value),
      'min_mae:%r'%(mae*max_value),
      'max_acc:%r'%(acc),
      'r2:%r'%(r2_score),
      'var:%r'%(var_score))

min_rmse:3.6910491946089006 min_mae:2.5546107 max_acc:0.7430389821529388 r2:0.8666488379240036 var:0.8672683238983154
